### **Objetivos**

Nesse notebook irei analisar o arquivo 'business.json'.

Nele temos dados de negócios, incluindo dados de localização, atributos, categorias entre outros.

Irei realizar a manipulação e tratamento dos dados, além de gerar novas features, para que possam ser utilizados como inputs do futuro modelo.


#### 1. Importando as bibliotecas 

In [1]:
import os
import pandas as pd
import numpy as np
import ast
from sklearn.preprocessing import LabelEncoder

#### 2. Realizando a leitura do arquivo "business.json" e criando o dataset 

In [2]:
# Criando o dataset e exibindo uma amostra do mesmo

# Definindo uma variável com o caminho para os arquivos JSON
path = os.path.abspath('..') + '\\INPUT'

# Realizando a leitura do arquivo "business.json" e criando o dataset
business = pd.read_json(path + '\\business.json', lines=True)

# Exibindo as primeiras linhas do dataset para confirmar se o arquivo foi lido corretamente
business.head(3)


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."


In [3]:
# Verificando detalhes do dataset (valores faltantes, tipos de dados e quantidade de registros)

business.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150346 entries, 0 to 150345
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   business_id   150346 non-null  object 
 1   name          150346 non-null  object 
 2   address       150346 non-null  object 
 3   city          150346 non-null  object 
 4   state         150346 non-null  object 
 5   postal_code   150346 non-null  object 
 6   latitude      150346 non-null  float64
 7   longitude     150346 non-null  float64
 8   stars         150346 non-null  float64
 9   review_count  150346 non-null  int64  
 10  is_open       150346 non-null  int64  
 11  attributes    136602 non-null  object 
 12  categories    150243 non-null  object 
 13  hours         127123 non-null  object 
dtypes: float64(3), int64(2), object(9)
memory usage: 16.1+ MB


* Observa-se que cada linha desse dataset contém informações referentes à um NEGÓCIO ÚNICO e que temos 150.346 negócios cadastrados.

* Nota-se também a presença de valores faltantes nas 3 últimas features do dataset (attributes, categories e hours).

#### 3. Data Wrangling + Feature Engineering

Decidi iniciar pelo tratamento da feature **"categories"**.

In [4]:
# Exibindo uma amostra do conteúdo da feature "categories"

pd.set_option('display.max_colwidth', None)
pd.DataFrame(business['categories'].head(10))

,categories
0,"Doctors, Traditional Chinese Medicine, Naturopathic/Holistic, Acupuncture, Health & Medical, Nutritionists"
1,"Shipping Centers, Local Services, Notaries, Mailbox Centers, Printing Services"
2,"Department Stores, Shopping, Fashion, Home & Garden, Electronics, Furniture Stores"
3,"Restaurants, Food, Bubble Tea, Coffee & Tea, Bakeries"
4,"Brewpubs, Breweries, Food"
5,"Burgers, Fast Food, Sandwiches, Food, Ice Cream & Frozen Yogurt, Restaurants"
6,"Sporting Goods, Fashion, Shoe Stores, Shopping, Sports Wear, Accessories"
7,"Synagogues, Religious Organizations"
8,"Pubs, Restaurants, Italian, Bars, American (Traditional), Nightlife, Greek"
9,"Ice Cream & Frozen Yogurt, Fast Food, Burgers, Restaurants, Food"


In [5]:
# Verificando o total de categorias ÚNICAS cadastradas

print("Total de categorias únicas:",business["categories"].nunique())

Total de categorias únicas: 83160


Cada linha contém várias categorias, separadas por vírgula e no formato string, associadas a um **negócio único**.

Por esse motivo observamos uma quantidade tão elevada de categorias únicas presentes nessa feature (83.160).

In [6]:
# Verificando as top 10 categorias que possuem mais negócios cadastrados

business['categories'].value_counts().head(10)

Beauty & Spas, Nail Salons    1012
Restaurants, Pizza             935
Nail Salons, Beauty & Spas     934
Pizza, Restaurants             823
Restaurants, Mexican           728
Restaurants, Chinese           708
Mexican, Restaurants           672
Chinese, Restaurants           651
Food, Coffee & Tea             508
Beauty & Spas, Hair Salons     493
Name: categories, dtype: int64

Podemos notar que existem inconsistências como 'Beauty & Spas, Nail Salons' e 'Nail Salons, Beauty & Spas' sendo identificados como categorias únicas somente pelo fato de estarem ordenados de forma diferente.

Irei "desmembrar" as strings para descobrir quantas categorias realmente únicas possuímos no dataset. Também colocarei em ordem alfabética para solucionar o problema identificado acima.

In [7]:
# Extraindo as categorias únicas, realizando a ordenação e inserindo a nova feature no dataframe

# Classificando as categorias nulas como 'Unknown'
business["categories"].fillna(value='Unknown', inplace=True)

unique_categories = [] #Lista que irá conter as categorias únicas

def category_processing(feature):

    """
    Realiza a identificação dos valores únicos e também realiza a ordenação.

    :param feature: pandas Series
    :return pandas Dataframe
    """

    processed_categories = [] #Lista que irá conter cada uma das categorias após tratamento

    # Percorrendo cada valor da feature 'categories'
    for category in feature:
        # Realizando o tratamento das strings (removendo espaços e ordenando)
        new_category = category.replace(' ','').split(',')
        new_category.sort()
        # Verificando se o valor não se encontra na lista 'unique_categories'
        # e adicionando à lista caso a condição seja satisfeita
        for item in new_category:
            if item not in unique_categories:
                unique_categories.append(item)
        # Realizando tratamentos adicionais após a ordenação
        # e adicionando à lista 'processed_categories'  
        new_category = str(new_category).replace('[','').replace(']','').replace("'",'')
        processed_categories.append(new_category)
    
    return pd.DataFrame(processed_categories)

# Executando a função de processamento e atualizando a feature 'categories' com os valores tratados
business['categories'] = category_processing(business['categories'])

In [8]:
# Verificando o total de categorias REALMENTE ÚNICAS cadastradas após o tratamento inicial

print('Total de categorias únicas encontadas:', len(unique_categories))

Total de categorias únicas encontadas: 1312


In [9]:
# Exibindo o total de categorias encontradas na feature "categories" (após ordenação dos valores)

print("Total de categorias presentes na feature 'categories': ",business['categories'].nunique())

Total de categorias presentes na feature 'categories':  44741


Mesmo após realizar a ordenação alfabética notamos um número elevado de categorias (44.741) pois elas podem conter quantidades diferentes para um mesmo negócio.

Ex: "Acupunture, Doctors" será diferente de "Acupunture, Doctors, Health&Medical".

Sendo assim, resolvi realizar uma espécie de OneHotEncoding manualmente, utilizando apenas as categorias únicas encontradas anteriormente (1.312 categorias).

In [10]:
# Processando novamente as categorias para criar as novas features no formato OneHotEncoding

def one_hot_categories(feature):

    """
    Cria novas features para as 'categories' no formato OneHotEncoding.

    :param feature: pandas Series
    :return pandas Dataframe
    """
    
    ohe_categories = []
    
    for category in feature:
        category = category.split(',')
        row_dict = {}
        for item in category:
            item = item.replace(' ','')
            item = item.replace("'",'')
            row_dict['category_' + item] = 1
        ohe_categories.append(row_dict)
    
    return pd.DataFrame(ohe_categories)

# Executando a função de processamento
df_categories = one_hot_categories(business['categories'])

# Preenchendo os valores nulos com zeros
df_categories.fillna(0, inplace=True)

Como resultado obtive mais 1.312 features (uma para cada categorira **realmente única**).

In [11]:
# Exibindo uma amostra do conteúdo após o tratamento

df_categories.head(5)

,category_Acupuncture,category_Doctors,category_Health&Medical,category_Naturopathic/Holistic,category_Nutritionists,category_TraditionalChineseMedicine,category_LocalServices,category_MailboxCenters,category_Notaries,category_PrintingServices,...,category_AviationServices,category_ConveyorBeltSushi,category_DialysisClinics,category_Makerspaces,category_NaturalGasSuppliers,category_Hospitalists,category_SerboCroatian,category_CeremonialClothing,category_BubbleSoccer,category_TradeFairs
0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Para **otimizar** o resultado do processo acima e **ocupar menos espaço na memória** resolvi converter os valores de todas as features geradas para o formato 'int8' (estavam no formato 'float64') pois as mesmas só contém os valores 0 e 1.

In [12]:
# Criando uma função para exibir o total de memória utilizada por um determinado Dataframe

def mem_usage(df):

    """
    Calcula o total de memória utilizada por um Dataframe

    :param df: pandas Dataframe
    :return float
    """

    amount_used = 0

    for i in df:
        amount_used += df[i].memory_usage(index=False)

    return amount_used * (10**-6) # Retornando o valor em Megabytes

In [13]:
print(f'Espaço utilizado antes da otimização: {mem_usage(df_categories)} Megabytes')

# Realizando a conversão dos valores do dataset de 'float64' para 'int8'
df_categories = df_categories.astype('int8')

print(f'Espaço utilizado após da otimização: {mem_usage(df_categories)} Megabytes')

Espaço utilizado antes da otimização: 1578.031616 Megabytes
Espaço utilizado após da otimização: 197.253952 Megabytes


Observe a redução drástica na quantidade de memória utilizada!!

In [14]:
# Concatenando as novas features ao dataframe

business = pd.concat([business, df_categories], axis=1)

Em seguida irei realizar o tratamento da feature **'attributes'**.

In [15]:
# Exibindo uma amostra do conteúdo da feature "attributes"

pd.DataFrame(business['attributes'].head(10))

,attributes
0,{'ByAppointmentOnly': 'True'}
1,{'BusinessAcceptsCreditCards': 'True'}
2,"{'BikeParking': 'True', 'BusinessAcceptsCreditCards': 'True', 'RestaurantsPriceRange2': '2', 'CoatCheck': 'False', 'RestaurantsTakeOut': 'False', 'RestaurantsDelivery': 'False', 'Caters': 'False', 'WiFi': 'u'no'', 'BusinessParking': '{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}', 'WheelchairAccessible': 'True', 'HappyHour': 'False', 'OutdoorSeating': 'False', 'HasTV': 'False', 'RestaurantsReservations': 'False', 'DogsAllowed': 'False', 'ByAppointmentOnly': 'False'}"
3,"{'RestaurantsDelivery': 'False', 'OutdoorSeating': 'False', 'BusinessAcceptsCreditCards': 'False', 'BusinessParking': '{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}', 'BikeParking': 'True', 'RestaurantsPriceRange2': '1', 'RestaurantsTakeOut': 'True', 'ByAppointmentOnly': 'False', 'WiFi': 'u'free'', 'Alcohol': 'u'none'', 'Caters': 'True'}"
4,"{'BusinessAcceptsCreditCards': 'True', 'WheelchairAccessible': 'True', 'RestaurantsTakeOut': 'True', 'BusinessParking': '{'garage': None, 'street': None, 'validated': None, 'lot': True, 'valet': False}', 'BikeParking': 'True', 'GoodForKids': 'True', 'Caters': 'False'}"
5,"{'BusinessParking': 'None', 'BusinessAcceptsCreditCards': 'True', 'RestaurantsAttire': 'u'casual'', 'OutdoorSeating': 'True', 'RestaurantsReservations': 'False', 'Caters': 'False', 'RestaurantsTakeOut': 'True', 'Alcohol': 'u'none'', 'Ambience': 'None', 'GoodForKids': 'True', 'RestaurantsPriceRange2': '1', 'ByAppointmentOnly': 'False', 'CoatCheck': 'False', 'DogsAllowed': 'False', 'RestaurantsTableService': 'False', 'RestaurantsGoodForGroups': 'True', 'RestaurantsDelivery': 'True', 'WiFi': 'u'no'', 'WheelchairAccessible': 'True', 'HasTV': 'True', 'HappyHour': 'False', 'DriveThru': 'True', 'BikeParking': 'False'}"
6,"{'BusinessAcceptsCreditCards': 'True', 'RestaurantsPriceRange2': '2', 'BikeParking': 'True', 'BusinessParking': '{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}'}"
7,None
8,"{'Caters': 'True', 'Alcohol': 'u'full_bar'', 'RestaurantsAttire': 'u'casual'', 'RestaurantsDelivery': 'False', 'RestaurantsTakeOut': 'True', 'HasTV': 'True', 'NoiseLevel': 'u'average'', 'BusinessAcceptsCreditCards': 'True', 'OutdoorSeating': 'True', 'BusinessParking': '{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}', 'Ambience': '{'romantic': False, 'intimate': False, 'touristy': False, 'hipster': False, 'divey': False, 'classy': False, 'trendy': False, 'upscale': False, 'casual': False}', 'RestaurantsPriceRange2': '1', 'GoodForKids': 'True', 'WiFi': 'u'free'', 'RestaurantsReservations': 'False', 'RestaurantsGoodForGroups': 'True'}"
9,"{'RestaurantsAttire': ''casual'', 'RestaurantsGoodForGroups': 'False', 'BusinessAcceptsCreditCards': 'True', 'OutdoorSeating': 'True', 'GoodForKids': 'True', 'Alcohol': 'u'none'', 'BusinessParking': '{'garage': False, 'street': False, 'validated': False, 'lot': False, 'valet': False}', 'DogsAllowed': 'False', 'RestaurantsTableService': 'False', 'ByAppointmentOnly': 'False', 'WiFi': 'u'no'', 'RestaurantsPriceRange2': '1', 'RestaurantsReservations': 'False', 'HasTV': 'True', 'RestaurantsDelivery': 'True', 'CoatCheck': 'False', 'Caters': 'False', 'RestaurantsTakeOut': 'True', 'DriveThru': 'True', 'HappyHour': 'False', 'WheelchairAccessible': 'True'}"


Perceba que cada negócio possui atributos únicos e esses estão inseridos em dicionários que, em alguns casos, possuem outros dicionários como valores.

Exemplo: 'BusinessParking': '{'garage': None, 'street': None, 'validated': None, 'lot': True, 'valet': False}'

Com isso terei um pouco mais de trabalho para extrair os atributos únicos e criar novas features, mas vamos em frente!

In [16]:
# Processando os atributos para criar as novas features no formato OneHotEncoding

# Classificando os atributos nulos como 'Unknown'
business["attributes"].fillna(value='Unknown', inplace=True)

def one_hot_attributes(feature):
    
    """
    Cria novas features para 'attributes' no formato OneHotEncoding

    :param feature: pandas Series
    :return pandas Dataframe
    """

    ohe_attributes = []
    subattribute_values = []

    for attribute in feature:
        row_dict = {}
        att_dict = {}
        if type(attribute) is dict:
            for key, value in attribute.items():
                if '{' in value:
                    value = ast.literal_eval(value)
                    for subkey, subvalue in value.items():
                        att_dict['subattribute_' + key + '_' + subkey] = subvalue
                    row_dict['attribute_' + key] = 1
                else:
                    row_dict['attribute_' + key] = value
        else:
            row_dict['attribute_' + attribute] = 1

        ohe_attributes.append(row_dict)
        subattribute_values.append(att_dict)
            
    attributes_df = pd.DataFrame(ohe_attributes)
    subattributes_df = pd.DataFrame(subattribute_values)

    return pd.concat([attributes_df,subattributes_df], axis=1)

# Executando a função de processamento
df_attributes = one_hot_attributes(business['attributes'])

# Preenchendo os valores nulos com zeros
df_attributes.fillna(0, inplace=True)

Com a função acima consegui "desmembrar" os atributos e criar algo parecido com um OneHotEncoding, porém notem que existem valores como "True" e "False" que poderiam ser exibidos como 1 e 0.

Também existem atributos com valores distintos (ex: attribute_RestaurantsPriceRange2 e attribute_WiFi).

Na sequência irei tratar desse fatores.

In [17]:
# Exibindo uma amostra do resultado do primeiro tratamento da feature "attributes"

df_attributes.head(5)

,attribute_ByAppointmentOnly,attribute_BusinessAcceptsCreditCards,attribute_BikeParking,attribute_RestaurantsPriceRange2,attribute_CoatCheck,attribute_RestaurantsTakeOut,attribute_RestaurantsDelivery,attribute_Caters,attribute_WiFi,attribute_BusinessParking,...,subattribute_HairSpecializesIn_kids,subattribute_HairSpecializesIn_perms,subattribute_HairSpecializesIn_asian,subattribute_DietaryRestrictions_dairy-free,subattribute_DietaryRestrictions_gluten-free,subattribute_DietaryRestrictions_vegan,subattribute_DietaryRestrictions_kosher,subattribute_DietaryRestrictions_halal,subattribute_DietaryRestrictions_soy-free,subattribute_DietaryRestrictions_vegetarian
0,True,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,True,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,False,True,True,2,False,False,False,False,u'no',1,...,0,0,0,0,0,0,0,0,0,0
3,False,False,True,1,0,True,False,True,u'free',1,...,0,0,0,0,0,0,0,0,0,0
4,0,True,True,0,0,True,0,False,0,1,...,0,0,0,0,0,0,0,0,0,0


In [18]:
# Processando novamente os atributos

def extra_attributes_processing(feature):

    """
    Realiza um novo tratamento da feature 'attributes'

    :param feature: pandas Series
    :return value
    """
    
    if isinstance(feature, str):
        if feature == 'None' or feature == 'False':
            return 0
        elif feature == 'True':
            return 1
        else:
            return feature.replace("'",'')
        
    elif isinstance(feature, bool):
        if feature == True:
            return 1
        else:
            return 0
        
    elif np.isnan(feature):
        return 0
    
    else:
        return feature
    
df_attributes2 = pd.DataFrame()

# Executando a função de processamento
for feature in df_attributes:
    df_attributes2[feature] = pd.DataFrame(df_attributes[feature].apply(extra_attributes_processing))

In [19]:
# Exibindo uma amostra do resultado do segundo tratamento da feature "attributes"

df_attributes2.head(5)

,attribute_ByAppointmentOnly,attribute_BusinessAcceptsCreditCards,attribute_BikeParking,attribute_RestaurantsPriceRange2,attribute_CoatCheck,attribute_RestaurantsTakeOut,attribute_RestaurantsDelivery,attribute_Caters,attribute_WiFi,attribute_BusinessParking,...,subattribute_HairSpecializesIn_kids,subattribute_HairSpecializesIn_perms,subattribute_HairSpecializesIn_asian,subattribute_DietaryRestrictions_dairy-free,subattribute_DietaryRestrictions_gluten-free,subattribute_DietaryRestrictions_vegan,subattribute_DietaryRestrictions_kosher,subattribute_DietaryRestrictions_halal,subattribute_DietaryRestrictions_soy-free,subattribute_DietaryRestrictions_vegetarian
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,1,2,0,0,0,0,uno,1,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,1,0,1,0,1,ufree,1,...,0,0,0,0,0,0,0,0,0,0
4,0,1,1,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


O processamento está quase completo, porém ainda temos features como "attribute_WiFi" que possuem valores no formato string (ex: 'uno', 'ufree').

Utilizei o código abaixo para identificar essas features e assim poderei realizar tratamentos específicos para cada caso conforme sua necessidade.

In [20]:
# Identificando as features que necessitam de tratamento especial / adicional

special_features = []
for feature in df_attributes2:
    for val in df_attributes2[feature].values:
        if val not in [0,1]:
            if feature not in special_features:
                special_features.append(feature)

special_features

['attribute_RestaurantsPriceRange2',
 'attribute_WiFi',
 'attribute_Alcohol',
 'attribute_RestaurantsAttire',
 'attribute_NoiseLevel',
 'attribute_Smoking',
 'attribute_BYOBCorkage',
 'attribute_AgesAllowed']

In [21]:
# Analisando o conteúdo de cada feature para decidir como serão os tratamentos

for feature in special_features:
    print(f'{feature} possui os valores {df_attributes2[feature].unique()}')

attribute_RestaurantsPriceRange2 possui os valores [0 '2' '1' '3' '4']
attribute_WiFi possui os valores [0 'uno' 'ufree' 'free' 'no' 'upaid' 'paid']
attribute_Alcohol possui os valores [0 'unone' 'ufull_bar' 'none' 'full_bar' 'ubeer_and_wine' 'beer_and_wine']
attribute_RestaurantsAttire possui os valores [0 'ucasual' 'casual' 'uformal' 'dressy' 'udressy' 'formal']
attribute_NoiseLevel possui os valores [0 'uaverage' 'uquiet' 'average' 'uloud' 'uvery_loud' 'quiet' 'very_loud'
 'loud']
attribute_Smoking possui os valores [0 'uno' 'uoutdoor' 'uyes' 'outdoor' 'no']
attribute_BYOBCorkage possui os valores [0 'yes_free' 'no' 'yes_corkage' 'uyes_free' 'uyes_corkage' 'uno']
attribute_AgesAllowed possui os valores [0 'u21plus' 'uallages' 'u18plus']


Resolvi utilizar a função "cat.codes" do Pandas para transformar cada categoria em um número de forma simples e rápida.

In [22]:
for feature in special_features:
    df_attributes2[feature] = df_attributes2[feature].astype('category')
    df_attributes2[feature] = df_attributes2[feature].cat.codes

Novamente irei otimizar o uso de memória pois os valores contidos nesse Dataframe estão entre 0 e 9 e não há necessidade de armazená-los como 'float64'.

In [23]:
print(f'Espaço utilizado antes da otimização: {mem_usage(df_attributes2)} Megabytes')

# Realizando a conversão dos valores do dataset de 'float64' para 'int8'
df_attributes2 = df_attributes2.astype('int8')

print(f'Espaço utilizado após da otimização: {mem_usage(df_attributes2)} Megabytes')

Espaço utilizado antes da otimização: 98.626976 Megabytes
Espaço utilizado após da otimização: 13.380794 Megabytes


In [24]:
# Concatenando as novas features ao dataframe

business = pd.concat([business, df_attributes2], axis=1)

Na próxima etapa irei realizar o tratamento da feature **'hours'**.

In [25]:
# Exibindo uma amostra do conteúdo da feature "hours"

business['hours'].head(10)

0                                                                                                                                                                       None
1                            {'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', 'Wednesday': '8:0-18:30', 'Thursday': '8:0-18:30', 'Friday': '8:0-18:30', 'Saturday': '8:0-14:0'}
2         {'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', 'Wednesday': '8:0-22:0', 'Thursday': '8:0-22:0', 'Friday': '8:0-23:0', 'Saturday': '8:0-23:0', 'Sunday': '8:0-22:0'}
3         {'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', 'Wednesday': '7:0-20:0', 'Thursday': '7:0-20:0', 'Friday': '7:0-21:0', 'Saturday': '7:0-21:0', 'Sunday': '7:0-21:0'}
4                                                 {'Wednesday': '14:0-22:0', 'Thursday': '16:0-22:0', 'Friday': '12:0-22:0', 'Saturday': '12:0-22:0', 'Sunday': '12:0-18:0'}
5           {'Monday': '0:0-0:0', 'Tuesday': '6:0-22:0', 'Wednesday': '6:0-22:0', 'Thursday': '6:0-22:0', 'Friday': '9:0-0:0', 'Saturda

Observa-se que para cada dia da semana temos o horário de abertura e fechamento como uma única string. Também temos valores nulos nessa feature.

Então resolvi novamente utilizar um processamento parecido com os anteriores para criar novas features.

In [26]:
# Processando os atributos para criar as novas features no formato OneHotEncoding

def one_hot_hours(feature):
    
    """
    Cria novas features para 'hours' no formato OneHotEncoding

    :param feature: pandas Series
    :return pandas Dataframe
    """

    ohe_hours = []

    for item in feature:
        row_dict = {}
        if item == None :
            row_dict['hours_None'] = 1
        else:
            for key, value in item.items():
                row_dict['hours_' + key] = value

        ohe_hours.append(row_dict)

    hours_df = pd.DataFrame(ohe_hours)
    return hours_df

# Executando a função de processamento
df_hours = one_hot_hours(business['hours'])

# Preenchendo os valores nulos com zeros
df_hours.fillna(0, inplace=True)

Como podemos observar abaixo, criamos features para cada dia da semana e inserimos os horários respectivos em cada uma delas. Porém dessa forma ainda será difícil utilizar essa informação em um futuro modelo.

In [27]:
# Exibindo uma amostra do resultado

df_hours.head(5)

,hours_None,hours_Monday,hours_Tuesday,hours_Wednesday,hours_Thursday,hours_Friday,hours_Saturday,hours_Sunday
0,1.0,0,0,0,0,0,0,0
1,0.0,0:0-0:0,8:0-18:30,8:0-18:30,8:0-18:30,8:0-18:30,8:0-14:0,0
2,0.0,8:0-22:0,8:0-22:0,8:0-22:0,8:0-22:0,8:0-23:0,8:0-23:0,8:0-22:0
3,0.0,7:0-20:0,7:0-20:0,7:0-20:0,7:0-20:0,7:0-21:0,7:0-21:0,7:0-21:0
4,0.0,0,0,14:0-22:0,16:0-22:0,12:0-22:0,12:0-22:0,12:0-18:0


Novamente irei utilizar a função "cat.codes" do Pandas para transformar cada item em um número de forma simples e rápida.

In [28]:
for feature in df_hours:
    df_hours[feature] = df_hours[feature].astype('category')
    df_hours[feature] = df_hours[feature].cat.codes

In [29]:
# Exibindo uma amostra do resultado

df_hours.head(5)

,hours_None,hours_Monday,hours_Tuesday,hours_Wednesday,hours_Thursday,hours_Friday,hours_Saturday,hours_Sunday
0,1,0,0,0,0,0,0,0
1,0,1,1192,1177,1234,1296,1171,0
2,0,1115,1205,1191,1249,1315,1205,1111
3,0,996,1074,1061,1115,1180,1072,999
4,0,0,0,402,536,328,322,298


In [30]:
# Concatenando as novas features ao dataframe

business = pd.concat([business, df_hours], axis=1)

O último tratamento que irei realizar será a conversão das features 'city' e 'state' utilizando LabelEncoder.

In [31]:
features = ['city','state']
df_city_state = pd.DataFrame()

for feature in features:
    le = LabelEncoder()
    le.fit(business[feature])
    le_results = le.transform(business[feature])    

    df_city_state['le_' + feature] = le_results

In [32]:
# Concatenando as novas features ao dataframe

business = pd.concat([business, df_city_state], axis=1)

In [33]:
# Dropando as features 'categories', 'attributes' e 'hours' pois foram geradas novas features para elas

business.drop(['categories','attributes','hours'], axis=1, inplace=True)

In [33]:
# Exportando o dataset final para um arquivo CSV

business.to_csv(path_or_buf= '../INPUT/business_EDITED.csv')

Como resultado final dessa etapa obtive um dataset com 1.422 features.

No decorrer do projeto poderei aplicar técnicas de redução de dimensionalidade ao dataset se houver necessidade.

>> [Próxima etapa - EDA](./3_Business_EDA.ipynb)